# Homework 5 RF Accuracy Improvement

This assignment is inspired by examples of Shan-Hung Wu from National Tsing Hua University.

Requirement: improve the accuracy per feature of the following code from 0.03 up to at least 0.40 and accuracy should be more than 0.92

Here are three hints:

    You can improve the ratio by picking out or "creating" several features.
    Tune hyperparameters
    The ratio can be improved from 0.03 up to 0.47.

In [43]:
from collections import defaultdict

import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import spearmanr
from scipy.cluster import hierarchy
from scipy.spatial.distance import squareform

from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score


# load the breast_cancer dataset
init_data = load_breast_cancer()
(X, y) = load_breast_cancer(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train)

# print(X.shape)

# TODO Select some features 
result = permutation_importance(clf, X_train, y_train, n_repeats=10, random_state=42)
perm_sorted_idx = result.importances_mean.argsort()

corr = spearmanr(X).correlation
corr = (corr + corr.T) / 2
np.fill_diagonal(corr, 1)

distance_matrix = 1 - np.abs(corr)
dist_linkage = hierarchy.ward(squareform(distance_matrix))
dendro = hierarchy.dendrogram(
    dist_linkage, labels=data.feature_names.tolist(), ax=ax1, leaf_rotation=90
)
dendro_idx = np.arange(0, len(dendro["ivl"]))

cluster_ids = hierarchy.fcluster(dist_linkage, 1, criterion="distance")
cluster_id_to_feature_ids = defaultdict(list)
for idx, cluster_id in enumerate(cluster_ids):
    cluster_id_to_feature_ids[cluster_id].append(idx)

selected_features = [v[0] for v in cluster_id_to_feature_ids.values()]

# TODO need 5 fold cross validation

cur_score = cross_val_score(clf, X, y, cv=5)
print(cur_score, '\n', selected_features)
print("5 fold cross validation score: {:.4f}".format(cur_score.mean()))


[0.92105263 0.93859649 0.96491228 0.95614035 0.96460177] 
 [0, 1, 4, 5, 11]
5 fold cross validation score: 0.9491


In [46]:
# The cross validation indicated that 4 and 11 are the most acurate features but 0 and 5 yield the highest accuracy result

selected_features = [0,5]

# TODO Tune parameters for RandomForestClassifier
X_train_sel = X_train[:, selected_features]
X_test_sel = X_test[:, selected_features]

clf_sel = RandomForestClassifier(n_estimators=100, random_state=42)
clf_sel.fit(X_train_sel, y_train)

# TODO Calculate Average accuracy score
accuracy = clf_sel.score(X_test_sel, y_test)

print(
    "Accuracy on test data with features removed: {:.4f}".format(
        accuracy
    )
)
# TODO Calculate Average (accuracy score/number of features)
print(
    "Accuracy per feature: {:.4f}".format(
        accuracy/len(selected_features)
    )
)




Accuracy on test data with features removed: 0.9202
Accuracy per feature: 0.4601


['mean radius' 'mean compactness']
